## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [2]:
%pip install -r requirements.txt
from sqlalchemy import create_engine, text, MetaData, select, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

"""
Extrai as classes da base de dados Chinook usando reflexão.
O metadata.reflect() lê automaticamente a estrutura do banco.
"""
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print('-'*25)

"""
Configura o objeto Base mapeando automaticamente os modelos ORM das tabelas.
O automap_base cria classes Python para cada tabela do banco.
"""
Base = automap_base(metadata=metadata)
Base.prepare()

print("\nModelos ORM disponíveis:", list(Base.classes.keys()))


  Using cached sqlalchemy-2.0.44-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
Using cached sqlalchemy-2.0.44-cp311-cp311-win_amd64.whl (2.1 MB)

   ---------------------------------------- 0/2 [greenlet]
   ---------------------------------------- 0/2 [greenlet]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   ---

In [3]:
"""
Exemplo de consulta na tabela Albums usando o ORM.
session.scalars() retorna uma sequência de objetos individuais.
"""
session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(f"AlbumId: {first_album.AlbumId}, Title: {first_album.Title}")
session.close()

AlbumId: 1, Title: For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [4]:
"""
Imprime os três primeiros registros da tabela tracks.
Usa limit(3) para restringir o número de resultados.
"""
session = Session(engine)

tracks = session.scalars(select(Base.classes.tracks).limit(3)).all()

print(f"{'TrackId':<10} {'Name':<45} {'Composer':<30}")
print("-"*85)
for track in tracks:
    composer = track.Composer if track.Composer else "Desconhecido"
    print(f"{track.TrackId:<10} {track.Name:<45} {composer:<30}")

session.close()


TrackId    Name                                          Composer                      
-------------------------------------------------------------------------------------
1          For Those About To Rock (We Salute You)       Angus Young, Malcolm Young, Brian Johnson
2          Balls to the Wall                             Desconhecido                  
3          Fast As a Shark                               F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman


In [6]:
"""
Imprime o nome da faixa e o título do álbum das primeiras 20 faixas.
Realiza JOIN entre tracks e albums através da chave estrangeira AlbumId.
"""
session = Session(engine)

query = (
    select(Base.classes.tracks.Name, Base.classes.albums.Title)
    .join(Base.classes.albums)
    .limit(20)
)

results = session.execute(query).all()

print(f"{'Track':<50} {'Álbum':<40}")
print("-"*90)
for track_name, album_title in results:
    print(f"{track_name:<50} {album_title:<40}")

session.close()

Track                                              Álbum                                   
------------------------------------------------------------------------------------------
For Those About To Rock (We Salute You)            For Those About To Rock We Salute You   
Put The Finger On You                              For Those About To Rock We Salute You   
Let's Get It Up                                    For Those About To Rock We Salute You   
Inject The Venom                                   For Those About To Rock We Salute You   
Snowballed                                         For Those About To Rock We Salute You   
Evil Walks                                         For Those About To Rock We Salute You   
C.O.D.                                             For Those About To Rock We Salute You   
Breaking The Rules                                 For Those About To Rock We Salute You   
Night Of The Long Knives                           For Those About To Rock We Sal

In [7]:
"""
Imprime as 10 primeiras vendas de faixas da tabela invoice_items.
Para essas vendas, mostra os nomes das faixas vendidas e a quantidade.
JOIN entre invoice_items e tracks para obter o nome das faixas.
"""
session = Session(engine)

query = (
    select(
        Base.classes.invoice_items.InvoiceLineId,
        Base.classes.tracks.Name,
        Base.classes.invoice_items.Quantity,
        Base.classes.invoice_items.UnitPrice
    )
    .join(Base.classes.tracks)
    .limit(10)
)

results = session.execute(query).all()

print(f"{'ID':<6} {'Faixa':<50} {'Qtd':<6} {'Preço':<10}")
print("-"*72)
for line_id, track_name, quantity, price in results:
    print(f"{line_id:<6} {track_name:<50} {quantity:<6} ${price:.2f}")

session.close()

ID     Faixa                                              Qtd    Preço     
------------------------------------------------------------------------
1      Balls to the Wall                                  1      $0.99
2      Restless and Wild                                  1      $0.99
3      Put The Finger On You                              1      $0.99
4      Inject The Venom                                   1      $0.99
5      Evil Walks                                         1      $0.99
6      Breaking The Rules                                 1      $0.99
7      Dog Eat Dog                                        1      $0.99
8      Overdose                                           1      $0.99
9      Love In An Elevator                                1      $0.99
10     Janie's Got A Gun                                  1      $0.99


In [8]:
"""
Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
Usa func.sum() para agregar as quantidades vendidas por faixa.
group_by() agrupa por TrackId para evitar duplicatas.
order_by(desc()) ordena do maior para o menor.
"""
session = Session(engine)

query = (
    select(
        Base.classes.tracks.Name,
        func.sum(Base.classes.invoice_items.Quantity).label('total_vendido')
    )
    .join(Base.classes.invoice_items, 
          Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.tracks.TrackId, Base.classes.tracks.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)

results = session.execute(query).all()

print(f"{'Pos':<5} {'Faixa':<55} {'Vendas':<10}")
print("-"*70)
for i, (track_name, total) in enumerate(results, 1):
    print(f"{i:<5} {track_name:<55} {total:<10}")

session.close()


Pos   Faixa                                                   Vendas    
----------------------------------------------------------------------
1     Balls to the Wall                                       2         
2     Inject The Venom                                        2         
3     Snowballed                                              2         
4     Overdose                                                2         
5     Deuces Are Wild                                         2         
6     Not The Doctor                                          2         
7     Por Causa De Você                                       2         
8     Welcome Home (Sanitarium)                               2         
9     Snowblind                                               2         
10    Cornucopia                                              2         


In [9]:
"""
Quem são os 10 artistas que mais venderam?
Realiza múltiplos JOINs: invoice_items -> tracks -> albums -> artists
Soma todas as quantidades vendidas de faixas de cada artista.
"""
session = Session(engine)

query = (
    select(
        Base.classes.artists.Name,
        func.sum(Base.classes.invoice_items.Quantity).label('total_vendas')
    )
    .select_from(Base.classes.invoice_items)
    .join(Base.classes.tracks, 
          Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .join(Base.classes.albums, 
          Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .join(Base.classes.artists, 
          Base.classes.albums.ArtistId == Base.classes.artists.ArtistId)
    .group_by(Base.classes.artists.ArtistId, Base.classes.artists.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)

results = session.execute(query).all()

print(f"{'Pos':<5} {'Artista':<45} {'Total de Vendas':<15}")
print("-"*65)
for i, (artist_name, total) in enumerate(results, 1):
    print(f"{i:<5} {artist_name:<45} {total:<15}")

session.close()


Pos   Artista                                       Total de Vendas
-----------------------------------------------------------------
1     Iron Maiden                                   140            
2     U2                                            107            
3     Metallica                                     91             
4     Led Zeppelin                                  87             
5     Os Paralamas Do Sucesso                       45             
6     Deep Purple                                   44             
7     Faith No More                                 42             
8     Lost                                          41             
9     Eric Clapton                                  40             
10    R.E.M.                                        39             


In [10]:
"""
Análise estatística complementar do banco Chinook.
Fornece insights adicionais sobre o banco de dados.
"""
session = Session(engine)

# Total de faixas no banco
total_tracks = session.scalar(select(func.count(Base.classes.tracks.TrackId)))

# Total de vendas realizadas
total_sales = session.scalar(select(func.sum(Base.classes.invoice_items.Quantity)))

# Receita total
total_revenue = session.scalar(
    select(func.sum(Base.classes.invoice_items.UnitPrice * Base.classes.invoice_items.Quantity))
)

# Número de artistas
total_artists = session.scalar(select(func.count(Base.classes.artists.ArtistId)))

print("ESTATÍSTICAS DO BANCO CHINOOK")
print("="*50)
print(f"Total de Faixas: {total_tracks:,}")
print(f"Total de Vendas (unidades): {total_sales:,}")
print(f"Receita Total: ${total_revenue:,.2f}")
print(f"Total de Artistas: {total_artists:,}")
print(f"Média de Vendas por Faixa: {total_sales/total_tracks:.2f}")

session.close()


ESTATÍSTICAS DO BANCO CHINOOK
Total de Faixas: 3,503
Total de Vendas (unidades): 2,240
Receita Total: $2,328.60
Total de Artistas: 275
Média de Vendas por Faixa: 0.64
